In [ ]:
from __future__ import print_function

%matplotlib widget

import numpy as np
import matplotlib.pylab as plt
import matplotlib as mpl
import os
import sys
sys.path.append('/home/ollie/nhutter/raslyboca/MITgcm/utils/python/MITgcmutils/MITgcmutils/')
import llc,mds
import xmitgcm
from mpl_toolkits.basemap import Basemap
import datetime
import re

from ipywidgets import interact, interactive, fixed, interact_manual, GridspecLayout, jslink
import ipywidgets as widgets

In [ ]:
def sq(field):
    return np.ma.masked_where(field==0,field)

def mosaic_llc(field):
    return np.vstack([np.hstack([np.vstack([np.rot90(field[i]) for i in [9,8,7]]),
                                 np.vstack([np.rot90(field[i]) for i in [12,11,10]]),
                                 np.vstack([field[i] for i in [0,1,2]]),
                                 np.vstack([field[i] for i in [3,4,5]])]),
                      np.hstack([np.rot90(field[6])*np.tri(90)[::-1,:],np.triu(np.rot90(field[6],k=2))*np.tri(90)[::-1,:],
                                 np.triu(np.rot90(field[6],k=-1)),np.zeros(field[6].shape)])])[30:315,:]

# Analysis of one run

## Read adjoint sensitivities

In [ ]:
#var_names = [i for i in os.listdir('run01')]

dir_w = widgets.Text(value='', placeholder='type directory name', 
                     description='Directory:',disabled=False)

upl_w = widgets.ToggleButton(value=False, description='Load data', disabled=False,
                             button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
                             tooltip='Description', icon='check')

def print_num_ts(ts_r,fl_s,ts_c):
    print('Number of time steps to load: %i' %(np.diff(ts_r)/ts_st))
    print('Number of files to load:      %i' %(np.diff(ts_r)/ts_st*len(fl_s)//ts_c))

def update_data_names(dirname,upload):
    global fl_select, ts_range, ts_comp_text, df, ts_st
    if dirname is not '':
        if os.path.isdir(dirname):
            var_names = np.unique([i.split('.')[0] for i in os.listdir(dirname) 
                                   if i.startswith('ADJ') and i.endswith('meta')])
            ts = [int(i.split('.')[1]) for i in os.listdir(dirname) 
                  if i.startswith(var_names[0]) and i.endswith('meta')]
        else:
            var_names = []
            
        if len(var_names)>0:
            ts_st = np.diff(np.sort(ts)).mean()
            if 'fl_select' in globals() and 'ts_range' in globals() and 'ts_comp_text' in globals():
                fl_select = widgets.SelectMultiple(options=var_names, rows=10,
                                                   description='Sensitivities',
                                                   value=fl_select.value)
                ts_range = widgets.IntRangeSlider(value=ts_range.value,min=min(ts),
                                                  max=max(ts), step=ts_st,
                                                  description='Time steps:',
                                                  orientation='horizontal')
                ts_comp_text = widgets.IntText(value=ts_comp_text.value,description='Read every i-th time step:')
            else:
                fl_select = widgets.SelectMultiple(options=var_names, rows=10,
                                                   description='Sensitivities')
                ts_range = widgets.IntRangeSlider(min=min(ts),max=max(ts), 
                                                  step=ts_st,
                                                  description='Time steps:',
                                                  orientation='horizontal')
                ts_comp_text = widgets.IntText(value=1,description='Ts compr.:')
            
            ts_range.layout.width='500px'
            n_ts = widgets.interactive_output(print_num_ts, {'ts_r': ts_range,
                                                             'fl_s': fl_select,
                                                             'ts_c': ts_comp_text})
            display(widgets.HBox([fl_select,widgets.VBox([ts_range,ts_comp_text,n_ts])]))
            if upl_w.value:
                print('Load the fields: ', fl_select.value)
                df = xmitgcm.open_mdsdataset(dirname, prefix=fl_select.value,
                                             iters=[i for i in range(ts_range.value[0],
                                                                     ts_range.value[1],
                                                                     int(np.diff(np.sort(ts)).mean())*ts_comp_text.value)],
                                             geometry='llc', llc_method='smallchunks',
                                             delta_t=3600,ref_date="1947-1-1 0:0:0")
                print('Finished reading fields: ', fl_select.value)
            
output = widgets.interactive_output(update_data_names, {'dirname': dir_w, 'upload':upl_w})
sl_elements = widgets.HBox([dir_w,upl_w])
display(sl_elements,output)

## Plot sensitivities

In [ ]:
# Function to plot sensitivity fields over time
if 'fig' in globals(): del fig

fl_dr = widgets.Dropdown(options=[i for i in df.data_vars.keys()], value=[i for i in df.data_vars.keys()][0], description='Field:')
fl_dr.layout.width='200px'
pr_dr = widgets.Dropdown(options=['Global','Arctic'], value='Arctic', description='Projection:')
pr_dr.layout.width='200px'
ts_sl = widgets.IntSlider(value=1, min=1, max=df.dims['time'], step=1, description='Time step:')
vm_sl = widgets.FloatLogSlider(value=10e0,base=10,min=-20,max=20,step=0.1,description='vmax:')
play = widgets.Play(value=1,min=1,max=df.dims['time'],step=1,description="Press play",disabled=False)
ts_box = widgets.IntText(value=ts_sl.value,description='Ts:')
sl_elements = widgets.VBox([widgets.HBox([fl_dr, pr_dr, vm_sl]), widgets.HBox([play, ts_sl, ts_box])])
widgets.jslink((play, 'value'), (ts_sl, 'value'))
widgets.jslink((ts_sl, 'value'), (ts_box, 'value'))

def update_plot(fl,ts,vm):
    pcm.set_array(sq(df[fl][-ts,6,:,:]).ravel())
    pcm.set_clim([-vm,vm])
    fig.canvas.draw()

def update_plot(fl,ts,vm,proj):
    global fig,ax,pcm
    if proj=='Arctic':
        ts_data = df[fl][-ts,6,:,:]
    elif proj=='Global':
        ts_data = mosaic_llc(df[fl][-ts,:,:,:])
    if 'fig' in globals() and ts_data.size==pcm.get_array().size:
        pcm.set_array(sq(ts_data).ravel())
        pcm.set_clim([-vm,vm])
        fig.canvas.draw()
    else:
        fig,ax=plt.subplots(1,1)
        pcm=ax.pcolormesh(sq(ts_data),vmax=vm,vmin=-vm,cmap=plt.get_cmap('RdBu_r'))
        plt.colorbar(pcm)

output = widgets.interactive_output(update_plot, {'fl': fl_dr, 'ts': ts_sl, 'vm': vm_sl, 'proj': pr_dr})
sl_elements.layout.width='700px'
#output.layout.width='400px'
display(sl_elements, output)

# Compare multiple runs in different directories

Note the different directories need to have output at the same time steps for this...

## Read adjoint sensitivities

In [ ]:
#var_names = [i for i in os.listdir('run01')]

dir_w = widgets.Text(value='', placeholder='type directory name', 
                     description='Directories:',disabled=False)

upl_w = widgets.ToggleButton(value=False, description='Load data', disabled=False,
                             button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
                             tooltip='Description', icon='check')

def print_num_ts(ts_r,fl_s,ts_c):
    print('Number of time steps to load: %i' %(np.diff(ts_r)/ts_st))
    print('Number of files to load:      %i' %(np.diff(ts_r)/ts_st*len(fl_s)//ts_c*len(read_dirs)))

def update_data_names(dirnames,upload):
    global fl_select, ts_range, ts_comp_text, dfs, ts_st, read_dirs, not_read_dirs
    if dirnames is not '':
        i_var_names = []; i_ts = []; not_read_dirs = []
        for idir,dirname in enumerate([i.strip() for i in re.split(',|;|',dirnames)]):
            if os.path.isdir(dirname):
                i_var_names.append(np.unique([i.split('.')[0] for i in os.listdir(dirname) 
                                              if i.startswith('ADJ') and i.endswith('meta')]))
                i_ts.append([int(i.split('.')[1]) for i in os.listdir(dirname) 
                             if i.startswith(i_var_names[idir][0]) and i.endswith('meta')])
            else:
                not_read_dirs.append(dirname)
                print('Warning: %s does not include ADJ* files' %dirname)

        read_dirs = [idir for idir in [i.strip() for i in re.split(',|;|',dirnames)] if idir not in not_read_dirs]
        var_names = [iname for iname in np.unique(np.concatenate(i_var_names)) if np.all([iname in idir for idir in i_var_names])]
        ts = [its for its in np.unique(np.concatenate(i_ts)) if np.all([its in idir for idir in i_ts])]

        if len(var_names)>0:
            ts_st = np.diff(np.sort(ts)).mean()
            if 'fl_select' in globals() and 'ts_range' in globals() and 'ts_comp_text' in globals():
                fl_select = widgets.SelectMultiple(options=var_names, rows=10,
                                                   description='Sensitivities',
                                                   value=fl_select.value)
                ts_range = widgets.IntRangeSlider(value=ts_range.value,min=min(ts),
                                                  max=max(ts), step=ts_st,
                                                  description='Time steps:',
                                                  orientation='horizontal')
                ts_comp_text = widgets.IntText(value=ts_comp_text.value,description='Read every i-th time step:')
            else:
                fl_select = widgets.SelectMultiple(options=var_names, rows=10,
                                                   description='Sensitivities')
                ts_range = widgets.IntRangeSlider(min=min(ts),max=max(ts), 
                                                  step=ts_st,
                                                  description='Time steps:',
                                                  orientation='horizontal')
                ts_comp_text = widgets.IntText(value=1,description='Ts compr.:')
            
            ts_range.layout.width='500px'
            n_ts = widgets.interactive_output(print_num_ts, {'ts_r': ts_range,
                                                             'fl_s': fl_select,
                                                             'ts_c': ts_comp_text})
            display(widgets.HBox([fl_select,widgets.VBox([ts_range,ts_comp_text,n_ts])]))
            if upl_w.value:
                dfs = {}
                for dirname in read_dirs:
                    print('Load the fields: %s in %s' %(fl_select.value,dirname))
                    df = xmitgcm.open_mdsdataset(dirname, prefix=fl_select.value,
                                                 iters=[i for i in range(ts_range.value[0],
                                                                         ts_range.value[1],
                                                                         int(np.diff(np.sort(ts)).mean())*ts_comp_text.value)],
                                                 geometry='llc', llc_method='smallchunks',
                                                 delta_t=3600,ref_date="1947-1-1 0:0:0")
                    dfs[dirname] = df
                    print('Finished reading fields: %s in %s' %(fl_select.value,dirname))
            
output = widgets.interactive_output(update_data_names, {'dirnames': dir_w, 'upload':upl_w})
sl_elements = widgets.HBox([dir_w,upl_w])
display(sl_elements,output)

## Plot sensitivities

In [ ]:
# Function to plot sensitivity fields over time
if 'fig' in globals(): del fig

dir_dr1 = widgets.Dropdown(options=[i for i in dfs.keys()], value=[i for i in dfs.keys()][0], description='Field 1:')
dir_dr1.layout.width='200px'
dir_dr2 = widgets.Dropdown(options=[i for i in dfs.keys()], value=[i for i in dfs.keys()][0], description='Field 2:')
dir_dr2.layout.width='200px'
fl_dr = widgets.Dropdown(options=[i for i in dfs[dir_dr1.value].data_vars.keys()], value=[i for i in dfs[dir_dr1.value].data_vars.keys()][0], description='Field:')
fl_dr.layout.width='200px'
pr_dr = widgets.Dropdown(options=['Global','Arctic'], value='Arctic', description='Projection:')
pr_dr.layout.width='200px'
ts_sl = widgets.IntSlider(value=1, min=1, max=dfs[dir_dr1.value].dims['time'], step=1, description='Time step:')
vm_sl = widgets.FloatLogSlider(value=1e12,base=10,min=6,max=20,step=0.1,description='vmax:')
vm_diff_sl = widgets.FloatLogSlider(value=1e2,base=10,min=1,max=12,step=0.1,description='vmax (diff):')
play = widgets.Play(value=1,min=1,max=dfs[dir_dr1.value].dims['time'],step=1,description="Press play",disabled=False)
ts_box = widgets.IntText(value=ts_sl.value,description='Ts:')
sl_elements = widgets.VBox([widgets.HBox([fl_dr, pr_dr, vm_sl]), widgets.HBox([dir_dr1,dir_dr2,vm_diff_sl]), widgets.HBox([play, ts_sl, ts_box])])
widgets.jslink((play, 'value'), (ts_sl, 'value'))
widgets.jslink((ts_sl, 'value'), (ts_box, 'value'))

def update_plot(di1,di2,fl,ts,vm,vm_diff,proj):
    global fig,ax,pcm1,pcm2,pcm3
    if proj=='Arctic':
        ts_data1 = dfs[di1][fl][-ts,6,:,:]
        ts_data2 = dfs[di2][fl][-ts,6,:,:]
    elif proj=='Global':
        ts_data1 = mosaic_llc(dfs[di1][fl][-ts,:,:,:])
        ts_data2 = mosaic_llc(dfs[di2][fl][-ts,:,:,:])
    if 'fig' in globals() and ts_data1.size==pcm1.get_array().size and ts_data2.size==pcm2.get_array().size:
        pcm1.set_array(sq(ts_data1).ravel())
        pcm1.set_clim([-vm,vm])
        pcm2.set_array(sq(ts_data2).ravel())
        pcm2.set_clim([-vm,vm])
        pcm3.set_array((sq(ts_data1)-sq(ts_data2)).ravel())
        pcm3.set_clim([-vm_diff,vm_diff])
        fig.canvas.draw()
    else:
        fig,ax=plt.subplots(1,3,sharex=True,sharey=True,figsize=(10,4))
        pcm1=ax[0].pcolormesh(sq(ts_data1),vmax=vm,vmin=-vm,cmap=plt.get_cmap('PRGn_r'))
        pcm2=ax[1].pcolormesh(sq(ts_data2),vmax=vm,vmin=-vm,cmap=plt.get_cmap('PRGn_r'))
        pcm3=ax[2].pcolormesh(sq(ts_data1)-sq(ts_data2),vmax=vm_diff,vmin=-vm_diff,cmap=plt.get_cmap('RdBu_r'))
        #plt.colorbar(pcm)

output = widgets.interactive_output(update_plot, {'di1': dir_dr1, 'di2': dir_dr2, 'fl': fl_dr, 'ts': ts_sl, 
                                                  'vm': vm_sl, 'vm_diff': vm_diff_sl, 'proj': pr_dr})
sl_elements.layout.width='700px'
#output.layout.width='400px'
display(sl_elements, output)